In [1]:
import pytorch_lightning as pl
pl.seed_everything(42)
import torch
from neuralforecast.data.tsdataset import WindowsDataset
from neuralforecast.data.tsloader import TimeSeriesLoader
from neuralforecast.experiments.utils import get_mask_dfs
from neuralforecast.data.utils import create_synthetic_tsdata

Global seed set to 42


In [70]:
Y_df, X_df, S_df = create_synthetic_tsdata(sort=True)

In [71]:
Y_df.head()

,unique_id,ds,y
0,uid_1,2020-12-31,1
0,uid_10,2020-12-22,1
1,uid_10,2020-12-23,2
2,uid_10,2020-12-24,3
3,uid_10,2020-12-25,4


In [72]:
X_df.shape

(2080, 4)

In [73]:
S_df.shape

(64, 2)

In [74]:
prediction_length = 5
encoder_length = 10
batch_size = 16  # set this to a value which your GPU can handle

In [75]:
train_dataset = WindowsDataset(Y_df=Y_df, X_df=X_df, S_df=S_df, 
                            #    mask_df=train_mask_df, f_cols=feat_config.time_varying_known_categoricals + feat_config.time_varying_known_reals,
                               input_size=encoder_length,
                               output_size=prediction_length,
                               # sample_freq=1,
                               # complete_windows=False,
                               verbose=False)

In [76]:
train_dataloader = TimeSeriesLoader(dataset=train_dataset,
                                batch_size=batch_size,
                                # n_windows=1024,
                                eq_batch_size=True,
                                shuffle=True)

In [77]:
batch = next(iter(train_dataloader))
print("\nsizes of x =")
for key, value in batch.items():
    print(f"\t{key} = {value.size()}")


sizes of x =
	S = torch.Size([16, 1])
	Y = torch.Size([16, 15])
	X = torch.Size([16, 2, 15])
	available_mask = torch.Size([16, 15])
	sample_mask = torch.Size([16, 15])
	idxs = torch.Size([16])


In [78]:
from neuralforecast.models.transformer.autoformer import Autoformer

In [79]:
# from neuralforecast.data.datasets.long_horizon import LongHorizon

# Y_df, X_df, S_df = LongHorizon.load(directory='./data', group='ETTm2')
# Y_df = Y_df.reset_index(drop=True)
# Y_df.loc[Y_df['unique_id']=='OT','y'] = Y_df[Y_df['unique_id']=='OT']['y'] + 100 #To obseve differences

In [80]:
# f_cols = X_df.drop(columns=['unique_id', 'ds']).columns.to_list()

In [81]:
mc_model = {}

mc_model['seq_len'] = encoder_length
mc_model['label_len'] = encoder_length//2
mc_model['pred_len'] = prediction_length
mc_model['output_attention'] = False
mc_model['enc_in'] = 4
mc_model['dec_in'] = 4
mc_model['d_model'] = 512
mc_model['c_out'] = 7
mc_model['embed'] = 'timeF'
mc_model['freq'] = 'h'
mc_model['dropout'] = 0.05
mc_model['factor'] = 1
mc_model['n_heads'] = 8
mc_model['d_ff'] = 2_048
mc_model['moving_avg'] = 25 
mc_model['activation'] = 'gelu'
mc_model['e_layers'] = 2 
mc_model['d_layers'] = 1
mc_model['loss_train'] = 'MAE'
mc_model['loss_hypar'] = 0.5
mc_model['loss_valid'] = 'MAE'
mc_model['learning_rate'] = 0.001
mc_model['lr_decay'] = 0.5
mc_model['weight_decay'] = 0.
mc_model['lr_decay_step_size'] = 2
mc_model['random_seed'] = 1

# # Dataset parameters
# mc_data = {}
# mc_data['mode'] = 'iterate_windows'
# mc_data['n_time_in'] = mc_model['seq_len']
# mc_data['n_time_out'] = mc_model['pred_len']
# mc_data['batch_size'] = 2
# mc_data['scaler'] = None
# mc_data['max_epochs'] = None
# mc_data['max_steps'] = 1
# mc_data['early_stop_patience'] = 20
# mc_data['normalizer_y'] = None
# mc_data['normalizer_x'] = None

len_val = 11_520
len_test = 11_520

In [82]:
# from neuralforecast.data.tsdataset import IterateWindowsDataset
# from neuralforecast.experiments.utils import create_datasets
# from torch.utils.data import DataLoader


# train_dataset, val_dataset, test_dataset = create_datasets(mc=mc_data,
#                                                                      S_df=None, 
#                                                                      Y_df=Y_df, X_df=X_df,
#                                                                      f_cols=f_cols,
#                                                                      ds_in_val=len_val,
#                                                                      ds_in_test=len_test)

# train_loader = DataLoader(dataset=train_dataset,
#                           batch_size=int(mc_data['batch_size']),
#                           shuffle=True,
#                           drop_last=True)

# val_loader = DataLoader(dataset=val_dataset,
#                         batch_size=int(mc_data['batch_size']),
#                         shuffle=False)

# test_loader = DataLoader(dataset=test_dataset,
#                          batch_size=int(mc_data['batch_size']),
#                          shuffle=False)

In [83]:
# batch = next(iter(train_dataloader))
# print("\nsizes of x =")
# for key, value in batch.items():
#     print(f"\t{key} = {value.size()}")

In [84]:
model = Autoformer(
    **mc_model
)

In [85]:
model = model.to("cuda")

In [86]:
for key in batch.keys():
    batch[key] = batch[key].to('cuda')

In [87]:
model(batch)

RuntimeError: Given groups=1, weight of size [512, 4, 3], expected input[1, 16, 12] to have 4 channels, but got 16 channels instead